# Robyn Budget Allocator Demo

This notebook demonstrates how to use the Python implementation of Robyn's budget allocator.
It shows how to:
1. Load and prepare data
2. Configure the allocator
3. Run optimization scenarios
4. Analyze and visualize results

## Step 1: Load Exported R Data

In [1]:
## Step 1: Setup and Import
import sys
import os
import pandas as pd
import numpy as np
from typing import Dict, Any, Union, List
import matplotlib.pyplot as plt

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

# Import necessary Robyn classes
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.modeling.pareto.pareto_optimizer import ParetoResult
from robyn.visualization.allocator_plotter import AllocationPlotter
from utils.data_mapper import (
    load_data_from_json,
    import_input_collect,
    import_output_collect,
    import_output_models,
)

2024-11-20 18:40:57,120 - robyn - INFO - Logging is set up to console only.


In [ ]:
# Load data from JSON exported from R
# raw_input_collect = load_data_from_json(
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/dh_Pareto_InputCollect.json"
# )
# raw_output_collect = load_data_from_json(
#     "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/dh_Allocator_OutputCollect.json"
# )

raw_input_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/yj_InputCollect.json"
)
raw_output_collect = load_data_from_json(
    "/Users/yijuilee/project_robyn/original/Robyn_original_2/Robyn/robyn_api/data/yj_OutputCollect.json"
)

# raw_input_collect = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_InputCollect.json"
# )
# raw_output_collect = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputCollect.json"
# )
# raw_output_models = load_data_from_json(
#     "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputModels.json"
# )

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_collect = import_output_collect(raw_output_collect)
# python_model_outputs = import_output_models(raw_output_models)

# Extract individual components
mmm_data = r_input_collect["mmm_data"]
featurized_mmm_data = r_input_collect["featurized_mmm_data"]
holidays_data = r_input_collect["holidays_data"]
hyperparameters = r_input_collect["hyperparameters"]

# Print data summary
print(f"Data loaded successfully:")
print(
    f"- Data timeframe: {mmm_data.data[mmm_data.mmmdata_spec.date_var].min()} to {mmm_data.data[mmm_data.mmmdata_spec.date_var].max()}"
)
print(
    f"- Number of paid media channels: {len(mmm_data.mmmdata_spec.paid_media_spends)}"
)
print(f"- Channels: {mmm_data.mmmdata_spec.paid_media_spends}")

## Step 2: Set up Budget Allocator

Initialize the budget allocator with the selected model and data.

In [ ]:
# select_model = r_output_collect["pareto_result"].pareto_solutions[
#     0
# ]  # Taking first solution as example
# print(f"Selected model: {select_model}")

In [ ]:
for i in raw_output_collect["clusters"]["models"]:
    print(i["solID"])

In [ ]:
# Override
select_model = "3_216_3"

## Step 3: Run Different Optimization Scenarios

### Scenario 1: Default Max Response

In [ ]:
print("Hyperparameters raw output:", hyperparameters)

In [ ]:
# Assuming `hyperparameters` is an instance of the `Hyperparameters` class

# Iterate over each channel and its hyperparameters
for channel, params in hyperparameters.hyperparameters.items():
    print(f"Channel: {channel}")
    print(f"  Thetas: {params.thetas}")
    print(f"  Shapes: {params.shapes}")
    print(f"  Scales: {params.scales}")
    print(f"  Alphas: {params.alphas}")
    print(f"  Gammas: {params.gammas}")
    print(f"  Penalty: {params.penalty}")
    print()

# Print other attributes of the Hyperparameters class
print(f"Adstock: {hyperparameters.adstock}")
print(f"Lambda: {hyperparameters.lambda_}")
print(f"Train Size: {hyperparameters.train_size}")

In [ ]:
from robyn.allocator_v2.entities.allocation_params import AllocatorParams
from robyn.allocator_v2.entities.allocation_result import (
    AllocationResult,
    OptimOutData,
    MainPoints,
)
from robyn.allocator_v2.entities.optimization_result import OptimizationResult
from robyn.allocator_v2.entities.constraints import Constraints
from robyn.allocator_v2.optimizer import BudgetAllocator
from robyn.allocator_v2.constants import (
    SCENARIO_MAX_RESPONSE,
    ALGO_SLSQP_AUGLAG,
    CONSTRAINT_MODE_EQ,
    DEFAULT_CONSTRAINT_MULTIPLIER,
    DATE_RANGE_ALL,
)


# Create allocator parameters matching R Example 1
allocator_params = AllocatorParams(
    scenario=SCENARIO_MAX_RESPONSE,
    total_budget=None,  # When None, uses total spend in date_range
    target_value=None,
    date_range="all",
    channel_constr_low=[0.7],  # Single value for all channels
    channel_constr_up=[1.2, 1.5, 1.5, 1.5, 1.5],  # Different values per channel
    channel_constr_multiplier=3.0,
    optim_algo="SLSQP_AUGLAG",
    maxeval=100000,
    constr_mode=CONSTRAINT_MODE_EQ,
    plots=True,
)


print("\nInitial constraints:")
for channel, low, up in zip(
    mmm_data.mmmdata_spec.paid_media_spends,
    [0.7] * len(mmm_data.mmmdata_spec.paid_media_spends),  # Expand single value
    [1.2, 1.5, 1.5, 1.5, 1.5],  # Per channel values
):
    print(f"{channel}: {low:.1f}x - {up:.1f}x")

# Initialize budget allocator
allocator = BudgetAllocator(
    mmm_data=mmm_data,
    hyperparameters=hyperparameters,
    pareto_result=r_output_collect["pareto_result"],
    select_model=select_model,
    params=allocator_params,
)

## Step 3: Run Optimization
allocation_result = allocator.optimize()

## Step 4: Analyze Results
print("\nOptimization Results Summary:")
print("-" * 50)
print(f"Model ID: {select_model}")
print(f"Scenario: {allocation_result.scenario}")
print(f"Use case: {allocation_result.usecase}")

results_df = pd.DataFrame(
    {
        "Channel": allocation_result.dt_optimOut.channels,
        "Initial Spend": allocation_result.dt_optimOut.init_spend_unit,
        "Optimized Spend": allocation_result.dt_optimOut.optm_spend_unit,
        "Spend Change %": (
            allocation_result.dt_optimOut.optm_spend_unit
            / allocation_result.dt_optimOut.init_spend_unit
            - 1
        )
        * 100,
        "Initial Response": allocation_result.dt_optimOut.init_response_unit,
        "Optimized Response": allocation_result.dt_optimOut.optm_response_unit,
        "Response Lift %": (
            allocation_result.dt_optimOut.optm_response_unit
            / allocation_result.dt_optimOut.init_response_unit
            - 1
        )
        * 100,
    }
)

print("\nDetailed Results:")
print(results_df.round(2))

# Print additional diagnostics
print("\nOptimization Parameters:")
print(f"Total budget: {allocator.constraints.budget_constraint:,.2f}")
print("Bound multiplier:", allocator_params.channel_constr_multiplier)
print("\nConstraint Violations:")
violations = np.sum(
    np.abs(allocation_result.dt_optimOut.optm_spend_unit - allocator.init_spend_unit)
)
print(f"Total allocation adjustment: {violations:,.2f}")

In [ ]:
print(allocation_result.dt_optimOut)
print(allocation_result.mainPoints)

In [ ]:
from robyn.allocator_v2.visualization.allocator_plotter import (
    AllocatorPlotter,
    PlotConfig,
)

# After getting allocation results:
plotter = AllocatorPlotter()

config = PlotConfig(
    title=f"Robyn Allocator Results: {select_model}",
    metric="ROAS" if mmm_data.mmmdata_spec.dep_var_type == "revenue" else "CPA",
    interval_type=mmm_data.mmmdata_spec.interval_type,
    model_id=select_model,
    # errors=f"Adj.R2: train = {rsq_train:.4f}, val = {rsq_val:.4f}",
    window_start=allocation_result.dt_optimOut.date_min[0],
    window_end=allocation_result.dt_optimOut.date_max[0],
    scenario=allocation_result.scenario,
)

plots = plotter.create_plots(
    allocation_result=allocation_result,
    config=config,
    export=False,
    # plot_folder="~/Desktop/robyn_plots",
)

# Display plots
for name, fig in plots.items():
    plt.figure(fig.number)
    plt.show()

### Scenario 2: Max Response with Custom Settings

### Scenario 3: Target Efficiency
Optimize allocation based on target ROI/CPA.

### Scenario 4: Custom Target Efficiency